In [44]:
#import the API and getpass
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
from getpass import getpass
import pandas as pd
import requests

#authenticate with your account (create a free one here: https://www.arcgis.com/home/createaccount.html)
gis = GIS("https://www.arcgis.com", "astrong_pnw", getpass())
print(gis)

········
GIS @ https://pnw.maps.arcgis.com


In [45]:
#Spreadsheet works: https://opendata.arcgis.com/datasets/ca7680fe3a734c6684c4b6f48e07548e_0.csv
#so just download this read it in pandas then upload it back up,,, should work 
# -*- coding: utf-8 -*-
url = 'https://opendata.arcgis.com/datasets/ca7680fe3a734c6684c4b6f48e07548e_0.csv'

#download csv from url and encode utf-8
with requests.Session() as s:
    download = s.get(url)
    download_decoded = download.content.decode('utf-8')

#write downloaded csv to file
f = open('data.csv', 'w', encoding='utf-8')
f.write(download_decoded)
f.close
print("csv downloaded")

csv downloaded


In [46]:
#now we can read this with pandas 
data = 'data.csv'
df = pd.read_csv(data)

df[:5]

,X,Y,GISOBJID,OBJ_CODE,OWNER,POLE_TYPE,COLOR,LAMP_TYPE,WATTAGE,FIXTURE,PLAN_NO,SERVICE_NO,COMMISSION_DATE,OBJECTID,ESRI_OID
0,-121.495857,38.678290,100043389,SLT-37281,DOT,ORN,YES,HPS,100.0,NaN,E-3527A,1643,2012-01-31T00:00:00.000Z,37281,4001
1,-121.496072,38.677849,100043018,SLT-37283,DOT,ORN,YES,HPS,100.0,NaN,E-3527A,1643,2012-01-31T00:00:00.000Z,37283,4002
2,-121.495849,38.677562,100043019,SLT-37284,DOT,ORN,YES,HPS,100.0,NaN,E-3527A,1643,2012-01-31T00:00:00.000Z,37284,4003
3,-121.495643,38.677119,100043020,SLT-37285,DOT,ORN,YES,HPS,100.0,NaN,E-3527A,1643,2012-01-31T00:00:00.000Z,37285,4004
4,-121.498325,38.676328,100043021,SLT-37286,DOT,ORN,YES,HPS,100.0,NaN,E-3527A,1643,2012-01-31T00:00:00.000Z,37286,4005


In [47]:
#Can we calculate energy consumption by total wats? http://www.lightsearch.com/resources/lightguides/formulas.html
watt = df['WATTAGE']
watt = pd.to_numeric(watt, errors='coerce')

#Get KwH from Watts
kwh = watt/1000
kwh[:5]

0    0.1
1    0.1
2    0.1
3    0.1
4    0.1
Name: WATTAGE, dtype: float64

In [48]:
#now we can calculate total energy consumption by mulitplying each KwH by the total hours of operation
energy_consumption = kwh * 3285
energy_consumption[:5]

0    328.5
1    328.5
2    328.5
3    328.5
4    328.5
Name: WATTAGE, dtype: float64

In [49]:
#add new field
df['E_CON'] = energy_consumption
df[:5]

,X,Y,GISOBJID,OBJ_CODE,OWNER,POLE_TYPE,COLOR,LAMP_TYPE,WATTAGE,FIXTURE,PLAN_NO,SERVICE_NO,COMMISSION_DATE,OBJECTID,ESRI_OID,E_CON
0,-121.495857,38.678290,100043389,SLT-37281,DOT,ORN,YES,HPS,100.0,NaN,E-3527A,1643,2012-01-31T00:00:00.000Z,37281,4001,328.5
1,-121.496072,38.677849,100043018,SLT-37283,DOT,ORN,YES,HPS,100.0,NaN,E-3527A,1643,2012-01-31T00:00:00.000Z,37283,4002,328.5
2,-121.495849,38.677562,100043019,SLT-37284,DOT,ORN,YES,HPS,100.0,NaN,E-3527A,1643,2012-01-31T00:00:00.000Z,37284,4003,328.5
3,-121.495643,38.677119,100043020,SLT-37285,DOT,ORN,YES,HPS,100.0,NaN,E-3527A,1643,2012-01-31T00:00:00.000Z,37285,4004,328.5
4,-121.498325,38.676328,100043021,SLT-37286,DOT,ORN,YES,HPS,100.0,NaN,E-3527A,1643,2012-01-31T00:00:00.000Z,37286,4005,328.5


In [55]:
#let's add this to our account with the python api 
#works if i do it with 'data', but having issues with 'new_data'
df.to_csv('new_data.csv', index=False)
new_data = 'new_data.csv'
item_prop = {'title':'Sac Streetlights Energy Consumption', 'type': 'CSV'}
csv_item = gis.content.add(item_prop, new_data)
print(f"item added to {gis}")

#publish
csv_item.publish()
print(f"item published to {gis}")

item added to GIS @ https://pnw.maps.arcgis.com
item published to GIS @ https://pnw.maps.arcgis.com
